In [ ]:
import pandas as pd

In [ ]:
pivot_file = "scf_2025_1.1.xlsx"

df = pd.read_excel(pivot_file, sheet_name="SCF 2025.1.1")

# we create a requirement node for each item in column C, and put the node name as the cell content and node framework as SCF
# we create a link from the requirement to the SCF framework
# for each column in AB to JR
# we create a framework for each column, framework name is column name and urn the slugified name
# per column, for each cell, we split the multiline content and create a requirement node for each one, the name is the id that we're getting, the framework is the column name
# we create a link for each requirement node to the framework as well

# now the last part is to create a link from each requirement detected in a cell when crawling frameworks

In [ ]:
import pandas as pd
from slugify import slugify
from openpyxl.utils import column_index_from_string
from neo4j import GraphDatabase

# Neo4j connection
uri = "bolt://localhost:7687"
user = "neo4j"
password = "badpass123"
driver = GraphDatabase.driver(uri, auth=(user, password))


def run_cypher(tx, query, **params):
    return tx.run(query, **params)


# 1) Load your pivot
pivot_file = "scf_2025_1.1.xlsx"
df = pd.read_excel(pivot_file, sheet_name="SCF 2025.1.1")

# 2) Determine column ranges
main_idx = column_index_from_string("C") - 1  # zero-based index of column C
start = column_index_from_string("AB") - 1
end = column_index_from_string("JR") - 1
framework_cols = df.columns[start : end + 1]

with driver.session() as sess:
    # --- Upsert the SCF framework ---
    sess.execute_write(
        run_cypher,
        """
        MERGE (f:Framework {urn: $urn})
        ON CREATE SET f.name = $name
        ON MATCH  SET f.name = $name
    """,
        urn="scf",
        name="SCF",
    )

    # --- Create Requirements from Column C and link to SCF ---
    parent_urn = "scf"
    for cell in df.iloc[:, main_idx].dropna().unique():
        raw_name = str(cell).strip()
        # namespace both urn and name with parent framework slug
        urn = f"{parent_urn}-{slugify(raw_name)}"
        name = f"{parent_urn}‑{raw_name}"
        sess.execute_write(
            run_cypher,
            """
            MERGE (r:Requirement {urn: $urn})
            ON CREATE SET r.name = $name, r.framework = $fw
            ON MATCH  SET r.name = $name
        """,
            urn=urn,
            name=name,
            fw="SCF",
        )
        sess.execute_write(
            run_cypher,
            """
            MATCH (r:Requirement {urn: $urn}), (f:Framework {urn: $fw_urn})
            MERGE (r)-[:PartOf]->(f)
        """,
            urn=urn,
            fw_urn=parent_urn,
        )

    # --- For each column AB→JR: upsert framework + requirements + links ---
    for col in framework_cols:
        fw_name = col.strip()
        fw_urn = slugify(fw_name)
        # upsert this column's framework
        sess.execute_write(
            run_cypher,
            """
            MERGE (f:Framework {urn: $urn})
            ON CREATE SET f.name = $name
            ON MATCH  SET f.name = $name
        """,
            urn=fw_urn,
            name=fw_name,
        )

        # for each non-null cell, split lines into IDs
        for raw in df[col].dropna():
            for part in str(raw).split("\n"):
                raw_id = part.strip()
                if not raw_id:
                    continue
                # namespace urn and name by framework
                urn = f"{fw_urn}-{slugify(raw_id)}"
                name = f"{fw_urn}‑{raw_id}"
                sess.execute_write(
                    run_cypher,
                    """
                    MERGE (r:Requirement {urn: $urn})
                    ON CREATE SET r.name = $name, r.framework = $fw_name
                    ON MATCH  SET r.name = $name
                """,
                    urn=urn,
                    name=name,
                    fw_name=fw_name,
                )
                sess.execute_write(
                    run_cypher,
                    """
                    MATCH (r:Requirement {urn: $urn}), (f:Framework {urn: $fw_urn})
                    MERGE (r)-[:PartOf]->(f)
                """,
                    urn=urn,
                    fw_urn=fw_urn,
                )

In [11]:
import pandas as pd
from slugify import slugify
from openpyxl.utils import column_index_from_string
import kuzu

# 1) Read the pivot and find columns
pivot_file = "scf_2025_1.1.xlsx"
df = pd.read_excel(pivot_file, sheet_name="SCF 2025.1.1")

main_idx = column_index_from_string("C") - 1
start, end = (column_index_from_string("AB") - 1, column_index_from_string("JR") - 1)
framework_cols = df.columns[start : end + 1]

# 2) Build CSV rows for frameworks, requirements, part‑of edges, AND the new requires edges
framework_rows = []
requirement_rows = []
partof_rows = []
requires_rows = []

# top‑level framework
framework_rows.append({"urn": "scf", "name": "SCF"})

# iterate rows once
for _, row in df.iterrows():
    # main requirement URN + name
    main_raw = row.iloc[main_idx]
    if pd.isna(main_raw):
        continue
    main_raw = str(main_raw).strip()
    main_urn = f"scf-{slugify(main_raw)}"
    main_name = f"scf‑{main_raw}"
    requirement_rows.append({"urn": main_urn, "name": main_name, "framework": "SCF"})
    partof_rows.append({"from": main_urn, "to": "scf"})

    # per‑column frameworks + sub‑requirements
    for col in framework_cols:
        fw_name = col.strip()
        fw_urn = slugify(fw_name)
        framework_rows.append({"urn": fw_urn, "name": fw_name})

        cell = row[col]
        if pd.isna(cell):
            continue
        for line in str(cell).split("\n"):
            sub = line.strip()
            if not sub:
                continue
            sub_urn = f"{fw_urn}-{slugify(sub)}"
            sub_name = f"{fw_urn}‑{sub}"
            requirement_rows.append(
                {"urn": sub_urn, "name": sub_name, "framework": fw_name}
            )
            # link sub→framework
            partof_rows.append({"from": sub_urn, "to": fw_urn})
            # ★ link main→sub
            requires_rows.append({"from": main_urn, "to": sub_urn})

# 3) Dump de‑duplicated CSVs
fr_df = pd.DataFrame(framework_rows).drop_duplicates(subset=["urn"], keep="first")
req_df = pd.DataFrame(requirement_rows).drop_duplicates(subset=["urn"], keep="first")
partof_df = pd.DataFrame(partof_rows).drop_duplicates(subset=["from", "to"])
requires_df = pd.DataFrame(requires_rows).drop_duplicates(subset=["from", "to"])

fr_df.to_csv("frameworks.csv", index=False)
req_df.to_csv("requirements.csv", index=False)
partof_df.to_csv("partof.csv", index=False)
requires_df.to_csv("requires.csv", index=False)

In [13]:
# 4) Kùzu: schema + batch loads (including the new Requires rel table)
db = kuzu.Database("./scf_db")
conn = kuzu.Connection(db)

conn.execute("""
  CREATE NODE TABLE IF NOT EXISTS Framework(
    urn STRING PRIMARY KEY,
    name STRING
  );
""")
conn.execute("""
  CREATE NODE TABLE IF NOT EXISTS Requirement(
    urn STRING PRIMARY KEY,
    name STRING,
    framework STRING
  );
""")
conn.execute("""
  CREATE REL TABLE IF NOT EXISTS PartOf(
    FROM Requirement TO Framework
  );
""")
# ← new relationship table:
conn.execute("""
  CREATE REL TABLE IF NOT EXISTS Requires(
    FROM Requirement TO Requirement
  );
""")

# Bulk‐load all four tables in one shot:
conn.execute('COPY Framework   FROM "frameworks.csv" (HEADER=true, PARALLEL=false);')
conn.execute('COPY Requirement FROM "requirements.csv" (HEADER=true, PARALLEL=false);')
conn.execute('COPY PartOf      FROM "partof.csv"   (HEADER=true, PARALLEL=false);')
conn.execute('COPY Requires    FROM "requires.csv" (HEADER=true, PARALLEL=false);')